In [1]:
#!/bin/env python3 
import os
import sys
import pandas as pd
from tabulate import tabulate

sys.path.insert(0,'src')
sys.path.insert(0,'../src')

from qwertyfolio import Transaction
from qwertyfolio import PortfolioManager
from qwertyfolio import parse_timestamp


from option_strategy_sim.chain_mgr import OptionChains, filter_by_value, filter_by_closest_value, get_first_value
from option_strategy_sim import OptionStrategy

port_file = 'portfolio.json'
transaction_file = 'transaction_log.csv' 
csvfile = './20250310_chains.csv'


def clear_files():
    if os.path.exists(port_file):
        os.remove(port_file)
    if os.path.exists(transaction_file):
        os.remove(transaction_file)




In [2]:
option_chains = OptionChains(csv=csvfile)
sym_chains = option_chains.get_symbol_chain('SPY')
sym_dte_chains = sym_chains.get_dte_chain(days_to_expiration=45, exact=False)
strangle = sym_dte_chains.strangle( quantity=-1, delta=.20, clear=True)

strangle


,symbol,underlying_symbol,strike_price,option_type,expiration_date,days_to_expiration,time,price,volatility,delta,...,bid_size,ask_size,open_interest,prev_day_volume,prev_day_close_price,mark,underlying_price,strike_percent,quote_date,quantity
0,SPY 250425C00597000,SPY,597.0,C,2025-04-25,46,1741616961,3.94,0.177,0.203,...,128,79,14,14,6.61,3.935,565.53,0.055647,2025-03-10,-1
1,SPY 250425P00525000,SPY,525.0,P,2025-04-25,46,1741616961,5.80,0.256,-0.195,...,96,44,30,41,3.87,5.785,565.53,-0.071667,2025-03-10,-1


In [3]:


# delta30 = filter_by_closest_value(sym_dte_chains.chain, 'delta', -.3)


In [4]:
sym_dte_chains.contract(symbol='SPY   250425C00597000', quantity=1, clear=True)
roll = sym_dte_chains.contract(option_type='C', quantity=-1, delta=0.3)
#txdate = parse_timestamp(str(get_first_value(strangle, 'time')))
# txdate = str(get_first_value(strangle, 'time'))
txdate = get_first_value(strangle, 'time')

clear_files()
portfolio = PortfolioManager(port_file, transaction_file)
# portfolio.update_cash_balance(10000)


tx = Transaction(timestamp=txdate, df=strangle)
print(f"\nTransaction 1 - {tx.chainid}")
portfolio.execute_open(tx)
portfolio.print_portfolio()

# print("THIS IS ROLL")
# print(roll)
# print()

# tx = Transaction(df=roll)
# print(f"\nTransaction 2 - {tx.chainid}")
# portfolio.execute_roll(tx)
# portfolio.print_portfolio()

# ironc = sym_dte_chains.strangle(quantity=-1, delta=.5)
# tx = Transaction(timestamp=txdate, df=ironc)
# print(f"\nTransaction 3 - {tx.chainid}")
# portfolio.execute_open(tx)
# portfolio.print_portfolio()




Transaction 1 - 2
____ Current Portfolio ____
Cash Balance: $972.00
+----+-----------------------+------------+---------+----------------------+--------------+---------------------+--------------+----------------------+---------------------------+----------------+---------+---------+---------+---------------------+--------------+-----------+--------------+
|    | symbol                |   quantity |   price |   average_open_price | asset_type   | underlying_symbol   |   multiplier |   days_to_expiration | expires_at                |   strike_price |   delta |   gamma |   theta | quote_date          | order_type   |   chainid |   roll_count |
|----+-----------------------+------------+---------+----------------------+--------------+---------------------+--------------+----------------------+---------------------------+----------------+---------+---------+---------+---------------------+--------------+-----------+--------------|
|  0 | SPY   250425C00597000 |         -1 |   3.935 |     

Error loading transactions from log file: transaction_log.csv - [Errno 2] No such file or directory: 'transaction_log.csv'


In [5]:
cols: list = "timestamp chainid symbol order_type quantity price strike_price expires_at days_to_expiration delta".split()
portfolio.transactions_log.show_columns = cols
portfolio.print_order_chains()



____ Order Chains ____
	Order chain: 2
+----+---------------------------+-----------+-----------------------+--------------+------------+---------+----------------+---------------------------+----------------------+---------+
|    | timestamp                 |   chainid | symbol                | order_type   |   quantity |   price |   strike_price | expires_at                |   days_to_expiration |   delta |
|----+---------------------------+-----------+-----------------------+--------------+------------+---------+----------------+---------------------------+----------------------+---------|
|  0 | 2025-03-10T14:29:21+00:00 |         2 | SPY   250425C00597000 | Sell to Open |         -1 |   3.935 |            597 | 2025-04-25T20:15:00+00:00 |                   46 |   0.203 |
|  1 | 2025-03-10T14:29:21+00:00 |         2 | SPY   250425P00525000 | Sell to Open |         -1 |   5.785 |            525 | 2025-04-25T20:15:00+00:00 |                   46 |  -0.195 |
|  2 | 2025-03-10T14:29:21